In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
! pip install optuna
import optuna
import sys
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForMaskedLM
import re
!pip install accelerate
!pip install transformers

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
import transformers
import accelerate
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)

Transformers version: 4.38.2
Accelerate version: 0.28.0


In [ ]:



# if len(sys.argv) < 3:
#     print("please give train vector path as 1st argument and test vector path in 2nd argument and ber model name as 3rd argument")
#     print("example - python Dnn.py bert_vectors_train_bert-base-uncased.pkl bert_vectors_val_bert-base-uncased.pkl bert-base-uncased")
#     print("available models are-")
#     print("bert-base-uncased   bert-base-cased   covid-twitter-bert   twhin-bert-base   SocBERT-base")
#     exit(1)

user_model_name_original = "bert-base-uncased"
train_file = "train_split.csv"
val_file = "val_split.csv"
# test_file = "test_split.csv"

print(f"AutoModelForSequenceClassification for {user_model_name_original}-")

AutoModelForSequenceClassification for bert-base-uncased-


In [ ]:
model_list = ["google-bert/bert-base-uncased", "google-bert/bert-base-cased", "digitalepidemiologylab/covid-twitter-bert","Twitter/twhin-bert-base", "sarkerlab/SocBERT-base"]
for model_name in model_list:
    if user_model_name_original == re.split(r'/', model_name)[1]:
        user_model_name = model_name
        break
print(user_model_name)

google-bert/bert-base-uncased


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(user_model_name)
model = AutoModelForSequenceClassification.from_pretrained(user_model_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
"""# task 1 , 2 , 3 (load)"""

try:
    # Load train and test data
    train_data = pd.read_csv(train_file)
    val_data = pd.read_csv(val_file)
    # test_data = pd.read_csv(test_file)

except:
    print("file path are invalid")
    # exit(1)


"""# Task 4"""

'# Task 4'

In [ ]:
train_data.head(5)

,tweet,label
0,india fights corona india report 90000 recover...,1
1,continuing good news daily reported death cont...,1
2,• 3 people currently hospital one auckland cit...,1
3,15/03 ncdc directly contacted twitter user men...,1
4,india confirmed case count cross ten lakh main...,0


In [ ]:
# Tokenize inputs for train, validation, and test sets with add_special_tokens=True
train_encodings = tokenizer(train_data["tweet"].tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_data["tweet"].tolist(), truncation=True, padding=True, max_length=512)
# test_encodings = tokenizer(test_data["tweet"].tolist(), truncation=True, padding=True, max_length=512)


In [ ]:
# Create PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_data["label"].tolist())
val_dataset = CustomDataset(val_encodings, val_data["label"].tolist())
# test_dataset = CustomDataset(test_encodings, test_data["label"].tolist())

"""## Basic Network"""

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

In [ ]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",             # Directory where model checkpoints and logs will be saved
    evaluation_strategy="epoch",                 # Evaluate the model after each epoch
    per_device_train_batch_size=8,              # Batch size per GPU during training
    per_device_eval_batch_size=8,               # Batch size for evaluation
    logging_dir="./logs",                        # Directory for storing logs
    logging_steps=500,                           # Log every 500 steps
    save_strategy="epoch",                       # Save a checkpoint at the end of each epoch
    save_total_limit=3,                          # Maximum number of checkpoints to save
    load_best_model_at_end=True,                 # Load the best model checkpoint at the end of training
    metric_for_best_model="eval_loss",           # Metric to use for determining the best model
    greater_is_better=False,                     # Smaller eval_loss is better
    # Add more configurations as needed
)

# Define Trainer with better configurations
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,            # Function to compute evaluation metrics
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.190000,0.162303,0.951887
2,0.099300,0.161056,0.956604
3,0.029600,0.202373,0.968868


TrainOutput(global_step=3180, training_loss=0.12280536732583676, metrics={'train_runtime': 2373.5778, 'train_samples_per_second': 10.718, 'train_steps_per_second': 1.34, 'total_flos': 6693545248358400.0, 'train_loss': 0.12280536732583676, 'epoch': 3.0})

In [ ]:
torch.save(model.state_dict(), "AutoModelForSequenceClassification_"+user_model_name_original+'.pth')

In [ ]:
# Create a folder in the root directory
!mkdir -p "/content/drive/My Drive/Models"
model_save_path = "/content/drive/MyDrive/Models/"+"AutoModelForSequenceClassification_"+user_model_name_original+'.pth'

# Save the model to the specified location in your Google Drive
torch.save(model.state_dict(), model_save_path)

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset

In [ ]:
test_file = "test_split.csv"

print(f"runEval AutoModelForSequenceClassification for {user_model_name_original}-")

runEval AutoModelForSequenceClassification for bert-base-uncased-


In [ ]:
user_model_name_original = "SocBERT-base"
model_list = ["google-bert/bert-base-uncased", "google-bert/bert-base-cased", "digitalepidemiologylab/covid-twitter-bert","Twitter/twhin-bert-base", "sarkerlab/SocBERT-base"]
for model_name in model_list:
    if user_model_name_original == re.split(r'/', model_name)[1]:
        user_model_name = model_name
        break
print(user_model_name)

sarkerlab/SocBERT-base


In [ ]:

try:
    test_data = pd.read_csv(test_file)

except:
    print("file path are invalid")
    # exit(1)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(user_model_name)
model_test = AutoModelForSequenceClassification.from_pretrained(user_model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/735k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/572M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sarkerlab/SocBERT-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_encodings = tokenizer(test_data["tweet"].tolist(), truncation=True, padding=True, max_length=512)


In [ ]:
# Create PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = CustomDataset(test_encodings, test_data["label"].tolist())


In [ ]:
model_test.load_state_dict(torch.load("AutoModelForSequenceClassification_"+user_model_name_original+'.pth'))

FileNotFoundError: [Errno 2] No such file or directory: 'AutoModelForSequenceClassification_SocBERT-base.pth'

In [ ]:
training_args_test = TrainingArguments(
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    output_dir="./results",
)

# Define the Trainer instance
trainer_test = Trainer(
    model=model_test,
    args=training_args_test,
)

In [ ]:
# Get predictions on the test dataset
predictions = trainer_test.predict(test_dataset)

# Extract predicted labels
predicted_labels = predictions.predictions.argmax(axis=1)

In [ ]:
predicted_labels.shape

In [ ]:
true_labels = test_dataset[:]['labels']

In [ ]:


# Calculate accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print("Test Accuracy:", test_accuracy)

In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')